# Model Merging 

I am now at the end of the pipeline that I defined in `N04_radio_info.ipynb`:

$Radio Message → [Sentiment Analysis] → [Intent Classification] → [Entity Extraction] → Structured Data$

The Structured data is neccesary to output a **standarized JSON format** that would be used by the logical agent in week 5 planning, alongside other models.

## Objetives of this notebook

The objectives and task to be done in this notebook are the following ones:

1. *Configure the mmodel paths*: first step, where I will configure the structure of the JSON.

2. *Load the pretrained models*: I will implement a function that will load the three elected models for making the radio analysis.

3. *Prediction functions*: for each model, it is necessary to implement a function where each models make their predictions.

4. *Audio Transcription*: first example usage, where I´ll use again Whisper for transcribing the radio message. Important for the urute if we want to directly pass the agent radio messages by radio.

5. *Integrating the pipeline into a single function*: this function will make the predictions of the three models.

6. *Analyzing text or audio examples*: two cells are going to be implemented for making the predictions in different types of radio formats.


---

## 1. Import Necessary Libraries

In [18]:
import json
import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
import spacy
import whisper
import torch
from transformers import BertForTokenClassification, BertConfig, BertTokenizer


## 2. Configuration

In this cell, I will configure the format of the JSON. It is necessary to specify:

- The model paths: 
    - ``../../outputs/week4/models/best_roberta_sentiment_model.pt``: best sentiment model developed.
    - ``../../outputs/week4/models/best_roberta_large_intent_model.pt``: best intent model developed.
    - ``../../outputs/week4/models/best_focused_bert_model.pt``: best NER model.
    - `Whisper turbo` model.
    
- Label mapppings of the models:
    - **sentiment**: positive, negative, neutral.
    - **intention**: INFORMATION, PROBLEM, ORDER, STRATEGY, WARNING, QUESTION.
    - **NER**: ACTION, SITUATION, INCIDENT, STRATEGY_INSTRUCTION, POSITION_CHANGE, PIT_CALL, TRACK_CONDITION, TECHNICAL_ISSUE, WEATHER.

- Demo input for making the correct structure.

In [19]:
# Configure model paths and settings - adjusted to match your specific models
CONFIG = {
    # Model paths
    "sentiment_model_path": "../../outputs/week4/models/best_roberta_sentiment_model.pt",
    "intent_model_path": "../../outputs/week4/models/best_roberta_large_intent_model.pt",
    "ner_model_path": "../../outputs/week4/models/best_focused_bert_model.pt",
    "whisper_model_size": "turbo",  # Using Whisper turbo 
    
    # Label mappings
    "sentiment_labels": ["positive", "negative", "neutral"],
    "intent_labels": ["INFORMATION", "PROBLEM", "ORDER",  "WARNING", "QUESTION"],
    
    # Entity mapping (SpaCy entity labels → output format)
    "entity_mapping": {
        "B-ACTION": "action",
        "I-ACTION": "action",
        "B-INCIDENT": "incident",
        "I-INCIDENT": "incident",
        "B-PIT_CALL": "pit",
        "I-PIT_CALL": "pit",
        "B-POSITION_CHANGE": "position",
        "I-POSITION_CHANGE": "position",
        "B-SITUATION": "situation",
        "I-SITUATION": "situation",
        "B-STRATEGY_INSTRUCTION": "strategy",
        "I-STRATEGY_INSTRUCTION": "strategy",
        "B-TECHNICAL_ISSUE": "technical",
        "I-TECHNICAL_ISSUE": "technical",
        "B-TRACK_CONDITION": "track",
        "I-TRACK_CONDITION": "track",
        "B-WEATHER": "weather",
        "I-WEATHER": "weather",
        "O": "O"
        },
    
    # Demo inputs
    "example_text": "Box this lap for softs, Hamilton is catching up"
}

print("Configuration loaded successfully")


Configuration loaded successfully


---

## 3. Loading Sentiment Model

In [20]:
def load_sentiment_model(device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Load the sentiment analysis model
    
    Args:
        device: Device to load the model on ('cuda' or 'cpu')
        
    Returns:
        Dictionary containing the loaded model and tokenizer
    """
    print(f"Loading sentiment model on {device}...")
    
    # Define base model for sentiment analysis
    base_model = "roberta-base"
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    
    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model, 
        num_labels=len(CONFIG["sentiment_labels"])
    )
    
    # Load fine-tuned weights
    model.load_state_dict(torch.load(CONFIG["sentiment_model_path"], map_location=device))
    
    # Move model to specified device and set to evaluation mode
    model.to(device)
    model.eval()
    
    print("Sentiment model loaded successfully!")
    
    return {
        "tokenizer": tokenizer,
        "model": model
    }



---

## 4. Loading Intent Model

In [21]:
def load_intent_model(device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Load the intent classification model
    
    Args:
        device: Device to load the model on ('cuda' or 'cpu')
        
    Returns:
        Dictionary containing the loaded model and tokenizer
    """
    print(f"Loading intent classification model on {device}...")
    
    # Define base model for intent classification
    base_model = "roberta-large"
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    
    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model, 
        num_labels=len(CONFIG["intent_labels"])
    )
    
    # Load fine-tuned weights
    model.load_state_dict(torch.load(CONFIG["intent_model_path"], map_location=device))
    
    # Move model to specified device and set to evaluation mode
    model.to(device)
    model.eval()
    
    print("Intent classification model loaded successfully!")
    
    return {
        "tokenizer": tokenizer,
        "model": model
    }



---

## 5. Loading NER model

In [22]:
def load_bert_ner_model(CONFIG, device="cuda" if torch.cuda.is_available() else "cpu"):
    
    # Create an initial congiguration based on the pretrained model, but with 19 labels
    config = BertConfig.from_pretrained(
        "dbmdz/bert-large-cased-finetuned-conll03-english",
        num_labels=len(CONFIG["entity_mapping"])
    )

    # Initilialize the model with this cofiguration
    model = BertForTokenClassification(config)

    # Load the weights of trained checkpoint with 19 labels
    model.load_state_dict(torch.load(CONFIG["ner_model_path"], map_location=device))
    model.to(device)
    model.eval()

    # Load the tokenizer of the pretrained model
    tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

    print("NER classification model loaded successfully!")
    return {
        "tokenizer": tokenizer,
        "model": model
    }


---

## 5. Sentiment Prediction Function

In [23]:
def predict_sentiment(text, model, tokenizer, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Predicts the sentiment of an F1 radio message.
    
    Args:
        text (str): Text of the radio message.
        model: Sentiment model.
        tokenizer: Tokenizer for preprocessing the text.
        device: Device for inference.
        
    Returns:
        dict: Dictionary with text, sentiment, and confidence.
    """
    # Tokenize the text
    encoded_text = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    # Move inputs to the device
    input_ids = encoded_text['input_ids'].to(device)
    attention_mask = encoded_text['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        pred_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][pred_class].item()
    
    # Map prediction to sentiment
    sentiments = ['positive', 'neutral', 'negative']
    
    # Return result as a dictionary
    result = {
        "text": text,
        "sentiment": sentiments[pred_class],
        "confidence": round(confidence * 100, 2)  # Convert to percentage
    }
    
    return result

In [24]:
# Load model and tokenizer
sentiment_result = load_sentiment_model()
sentiment_model = sentiment_result["model"]
sentiment_tokenizer = sentiment_result["tokenizer"]

# Verify that they loaded correctly
print(f"Tokenizer type: {type(sentiment_tokenizer)}")
print(f"Model type: {type(sentiment_model)}")


Loading sentiment model on cuda...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\victo\AppData\Local\Temp\ipykernel_10876\1002156124.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

Sentiment model loaded successfully!
Tokenizer type: <class 'transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast'>
Model type: <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>


In [25]:
# Example radio message
radio_message = "Great move Oscar"

# Predict sentiment
prediction = predict_sentiment(radio_message, sentiment_model, sentiment_tokenizer)
print(f"Text: '{prediction['text']}'")
print(f"Sentiment: {prediction['sentiment']}")
print(f"Confidence: {prediction['confidence']}%")

Text: 'Great move Oscar'
Sentiment: positive
Confidence: 59.62%


---

## 6. Intent Prediction Function

In [26]:
def predict_intent(text, model, tokenizer, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Predicts the intent of an F1 radio message.
    
    Args:
        text (str): Text of the radio message.
        model: Intent classification model.
        tokenizer: Tokenizer for preprocessing the text.
        device: Device for inference.
        
    Returns:
        dict: Dictionary with text, predicted intent, and confidence.
    """
    # Tokenize the text
    encoded_text = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    # Move inputs to the device
    input_ids = encoded_text['input_ids'].to(device)
    attention_mask = encoded_text['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
        pred_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][pred_class].item()
    
    # Map prediction to intent (assuming you have defined this in your CONFIG)
    intents = CONFIG["intent_labels"]
    
    # Return result as a dictionary
    result = {
        "text": text,
        "intent": intents[pred_class],
        "confidence": round(confidence * 100, 2)  # Convert to percentage
    }
    
    return result




In [27]:
# # Load intent model and tokenizer
# intent_result = load_intent_model()
# intent_model = intent_result["model"]
# intent_tokenizer = intent_result["tokenizer"]

# # Verify that they loaded correctly
# print(f"Tokenizer type: {type(intent_tokenizer)}")
# print(f"Model type: {type(intent_model)}")


In [28]:
# # Example radio message
# radio_message = "Possible debris turn 7"

# # Predict intent
# prediction = predict_intent(radio_message, intent_model, intent_tokenizer)
# print(f"Text: '{prediction['text']}'")
# print(f"Intent: {prediction['intent']}")
# print(f"Confidence: {prediction['confidence']}%")

## 7. NER Prediction Function

In [29]:
# # Load intent model and tokenizer
# ner_result = load_bert_ner_model(CONFIG)
# ner_model = ner_result["model"]
# ner_tokenizer = ner_result["tokenizer"]

# # Verify that they loaded correctly
# print(f"Tokenizer type: {type(ner_tokenizer)}")
# print(f"Model type: {type(ner_model)}")

In [30]:

# radio_message = "Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment."


# result = analyze_f1_radio(radio_message)


---

I created a `NLP_utils` folder, where there are python archives with the functions developed in previous notebooks. Thanks to this, I can import the prediction function for the NER model previously developed on `N05_ner_models`.

In [31]:
import json
import os
from datetime import datetime

def analyze_radio_message(radio_message):
    """
    Analyzes a radio message using sentiment, intent, and NER models.
    
    Args:
        radio_message (str): The radio message to analyze.
        
    Returns:
        str: The path to the generated JSON file.
    """
    import sys
    from io import StringIO
    
    # Redirect stdout to suppress model loading messages
    original_stdout = sys.stdout
    sys.stdout = StringIO()
    
    # Load sentiment model
    sentiment_result = load_sentiment_model()
    sentiment_model = sentiment_result["model"]
    sentiment_tokenizer = sentiment_result["tokenizer"]
    
    # Load intent model
    intent_result = load_intent_model()
    intent_model = intent_result["model"]
    intent_tokenizer = intent_result["tokenizer"]
    
    # Load NER model
    ner_result = load_bert_ner_model(CONFIG)
    ner_model = ner_result["model"]
    ner_tokenizer = ner_result["tokenizer"]
    
    # Execute NER notebook (assuming this is needed)
    from NLP_utils.N05_ner_models import analyze_f1_radio
    
    sys.stdout = original_stdout  # Restore stdout
    
    # Get predictions
    sentiment_prediction = predict_sentiment(radio_message, sentiment_model, sentiment_tokenizer)
    intent_prediction = predict_intent(radio_message, intent_model, intent_tokenizer)
    ner_entities = analyze_f1_radio(radio_message)
    
    # Structure response as a dictionary
    response = {
        "message": radio_message,
        "analysis": {
            "sentiment": sentiment_prediction["sentiment"],
            "intent": intent_prediction["intent"],
            "entities": ner_entities
        }
    }
    
    # Create a unique filename based on current date and time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Format: YYYYMMDD_HHMMSS
    json_filename = f"../../outputs/week4/json/radio_analysis_{timestamp}.json"
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(json_filename), exist_ok=True)
    
    # Save response as a JSON file
    with open(json_filename, "w") as json_file:
        json.dump(response, json_file, indent=2)
    
    print(f"Analysis saved to {json_filename}")
    
    return json_filename  # Return the filename for reference


In [32]:
def transcribe_audio(audio_path):
    """
    Transcribe F1 team radio audio using the Whisper model.
    
    Args:
        audio_path (str): Path to the audio file
        
    Returns:
        str: Transcribed text
    """
    print("Loading Whisper model...")
    model = whisper.load_model("turbo")  
    
    print(f"Transcribing audio: {audio_path}")
    result = model.transcribe(audio_path)
    
    print("Transcription completed.")
    return result["text"]

# Path to example audio file
audio_path = "../../f1-strategy/data/audio/driver_(1,)/driver_(1,)_belgium_radio_39.mp3"



In [33]:
# Transcribe the audio
transcribed_text = transcribe_audio(audio_path)
print("\nTranscribed Text:")
print(transcribed_text)
json_filename = analyze_radio_message(transcribed_text)

Loading Whisper model...


c:\Users\victo\miniconda3\envs\f1_strat_manager\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loca

Transcribing audio: ../../f1-strategy/data/audio/driver_(1,)/driver_(1,)_belgium_radio_39.mp3
Transcription completed.

Transcribed Text:
 So don't forget Max, use your head please. Are we both doing it or what? You just follow my instruction. No, I want to know if both cars do it. Max, please follow my instruction and trust it. Thank you.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\victo\AppData\Local\Temp\ipykernel_10876\1002156124.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u


Analyzing message: " So don't forget Max, use your head please. Are we both doing it or what? You just follow my instruction. No, I want to know if both cars do it. Max, please follow my instruction and trust it. Thank you."

Detected entities:
  ACTION:
    • "use"
    • "follow my instruction."
    • "please"
    • "follow my instruction and trust it."
  SITUATION:
    • "Are we both doing it or what?"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250331_134227.json


In [34]:
# Example radio message
radio_message = "Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment."

# Call fubction and obstain resulting JSON
json_filename = analyze_radio_message(radio_message)

# Print JSON Path
print(f"Analysis stored, path: {json_filename}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\victo\AppData\Local\Temp\ipykernel_10876\1002156124.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u


Analyzing message: "Max, we've currently got yellows in turn 7. Ferrari in the wall, no? Yes, that's Charles stopped. We are expecting the potential of an aborted start, but just keep to your protocol at the moment."

Detected entities:
  ACTION:
    • "keep to your protocol at the moment."
  INCIDENT:
    • "Ferrari in the wall,"
    • "that's Charles stopped."
  SITUATION:
    • "we've currently got yellows in turn 7."
    • "We are expecting the potential of an aborted start,"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250331_134233.json
Analysis stored, path: ../../outputs/week4/json/radio_analysis_20250331_134233.json


# 🏁 NLP Pipeline Summary: Model Integration

## What We've Accomplished

In this notebook, we successfully built a complete NLP pipeline for analyzing F1 team radio messages, completing the workflow:

`Radio Message → [Sentiment Analysis] → [Intent Classification] → [Entity Extraction] → Structured Data`

## Key Components Implemented

1. **Model Configuration**:
   - Defined paths for our pretrained models
   - Configured label mappings for sentiment, intent, and entity classes
   - Set up a standardized JSON output format

2. **Model Loading Functions**:
   - Created functions to load all three specialized models:
     - RoBERTa-base for sentiment analysis
     - RoBERTa-large for intent classification
     - BERT-large for named entity recognition (NER)

3. **Prediction Functions**:
   - Implemented dedicated functions for each analysis type
   - Created standardized output formats for each model

4. **Integration Pipeline**:
   - Combined all models into a single `analyze_radio_message()` function
   - Created a workflow that processes raw text through all models
   - Included Whisper transcription support for audio inputs

5. **JSON Output**:
   - Developed a structured output format containing:
     - Original message
     - Sentiment (positive, negative, neutral)
     - Intent (INFORMATION, PROBLEM, ORDER, WARNING, QUESTION)
     - Named entities with their types and positions

## Next Steps

This pipeline is now ready to be integrated with the logical agent system from week 5, providing rich structured data from team radio communications that can be used for strategic decision-making in our F1 strategy system.